<a href="https://colab.research.google.com/github/ggvet/fintech_analysis/blob/main/Segmentazione_della_clientela_per_campagne_di_marketing_mirate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Panoramica del Progetto: Segmentazione della Clientela per Campagne di Marketing Mirate**

**Obiettivo:** Segmentare la base clienti delle carte di credito di FinTech Solutions S.p.A. in gruppi distinti sulla base del loro comportamento transazionale. Questa segmentazione permetterà la creazione di campagne di marketing personalizzate, volte ad aumentare il coinvolgimento dei clienti, massimizzare il ritorno sull'investimento e migliorare il valore del ciclo di vita del cliente (customer lifetime value).

**Dataset:** L'analisi si basa su un dataset di circa 8.950 titolari di carte di credito relativo a un periodo di 12 mesi, che descrive in dettaglio le loro abitudini di spesa, il comportamento di pagamento e l'utilizzo del credito.

**Metodologia:** Il progetto segue un flusso di lavoro strutturato di data science:

*   **Analisi Esplorativa dei Dati (EDA):** Ispezione iniziale dei dati per comprenderne la struttura, le distribuzioni e identificare potenziali problemi come valori mancanti o outlier.

*   **Preprocessing dei Dati:** Pulizia e preparazione dei dati per la modellazione, che include la gestione dei valori mancanti e la scalatura delle feature per garantirne la comparabilità.

*   **Riduzione della Dimensionalità con PCA:** Applicazione dell'Analisi delle Componenti Principali (PCA) per ridurre la complessità del dataset, identificare i principali pattern comportamentali e facilitare la visualizzazione.

*   **Validazione del Modello di Clustering:** Utilizzo di molteplici metriche di validazione (Metodo del Gomito, Silhouette Score, Indice di Calinski-Harabasz, Indice di Davies-Bouldin) per determinare il numero ottimale di segmenti di clientela.

*   **Clustering con K-Means:** Applicazione dell'algoritmo K-Means per raggruppare i clienti nel numero ottimale di cluster identificato nella fase precedente.

*   **Profilazione e Interpretazione dei Cluster:** Analisi delle caratteristiche di ogni cluster per creare "personas" di clienti distinte e comprendere i loro specifici comportamenti finanziari.

*   **Raccomandazioni Strategiche:** Sviluppo di strategie di marketing azionabili e adattate alle esigenze e ai comportamenti specifici di ogni segmento di clientela.

Questo processo end-to-end garantisce che la segmentazione finale non sia solo statisticamente robusta, ma anche commercialmente rilevante e direttamente applicabile agli obiettivi di business di FinTech Solutions S.p.A.

### **1. Caricamento e Ispezione Iniziale dei Dati**

Il primo passo consiste nel caricare il dataset e nell'eseguire un'analisi preliminare per comprenderne le caratteristiche di base, tra cui dimensioni, tipi di dato e utilizzo della memoria. Questo passaggio fornisce una comprensione fondamentale prima di procedere con un'analisi più approfondita.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from scipy import stats
import warnings

# Configura gli stili di visualizzazione e le opzioni di visualizzazione di pandas
warnings.filterwarnings('ignore')
plt.style.use('default')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.3f}'.format)

# Clona il repository e carica il dataset
!git clone https://github.com/ggvet/fintech_analysis.git
df = pd.read_csv('fintech_analysis/credit_card_customers.csv')

# Mostra le informazioni di base sul DataFrame caricato
print("--- Panoramica Iniziale del Dataset ---")
print(f"Dimensioni: {df.shape[0]:,} righe × {df.shape[1]} colonne")
print(f"Utilizzo Memoria: {df.memory_usage().sum() / 1024**2:.2f} MB\n")

# Mostra un riepilogo dei tipi di dato e dei valori mancanti
print("--- Tipi di Dato e Valori Mancanti ---")
df.info()

# Mostra le prime righe per comprendere la struttura dei dati
print("\n--- Prime 5 Righe del Dataset ---")
print(df.head())

fatal: destination path 'fintech_analysis' already exists and is not an empty directory.
--- Panoramica Iniziale del Dataset ---
Dimensioni: 8,950 righe × 18 colonne
Utilizzo Memoria: 1.23 MB

--- Tipi di Dato e Valori Mancanti ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUST_ID                           8950 non-null   object 
 1   BALANCE                           8950 non-null   float64
 2   BALANCE_FREQUENCY                 8950 non-null   float64
 3   PURCHASES                         8950 non-null   float64
 4   ONEOFF_PURCHASES                  8950 non-null   float64
 5   INSTALLMENTS_PURCHASES            8950 non-null   float64
 6   CASH_ADVANCE                      8950 non-null   float64
 7   PURCHASES_FREQUENCY               8950 non-null   float64
 8   ONEOFF_PURCHASES_FREQUENCY

**Risultati Iniziali:**

*   Il dataset è composto da 8.950 clienti e 18 feature.
*   `CUST_ID` è un identificatore univoco e sarà escluso dal processo di modellazione.
*   Sono presenti valori mancanti in `CREDIT_LIMIT` (1 record) e `MINIMUM_PAYMENTS` (313 record), che richiedono di essere gestiti.
*   Tutte le feature, ad eccezione di `CUST_ID`, sono di tipo numerico, rendendole idonee per gli algoritmi di clustering.

### **2. Analisi Esplorativa dei Dati (EDA) e Pulizia dei Dati**

Questa fase si concentra su un'analisi più approfondita dei dati. Analizzeremo la distribuzione statistica di ogni variabile per identificare l'asimmetria (*skewness*) e la presenza di *outlier*. Successivamente, verranno gestiti i valori mancanti rilevati durante l'ispezione iniziale. Data la natura tipicamente asimmetrica dei dati finanziari, l'utilizzo della mediana per l'imputazione rappresenta una strategia più robusta rispetto alla media, in quanto è meno sensibile ai valori estremi.

In [6]:
# --- Gestione dei Valori Mancanti ---
# Imputa i valori mancanti di MINIMUM_PAYMENTS con la mediana, poiché la distribuzione è asimmetrica.
median_min_payments = df['MINIMUM_PAYMENTS'].median()
df['MINIMUM_PAYMENTS'].fillna(median_min_payments, inplace=True)

# Imputa il singolo valore mancante di CREDIT_LIMIT con la mediana.
median_credit_limit = df['CREDIT_LIMIT'].median()
df['CREDIT_LIMIT'].fillna(median_credit_limit, inplace=True)

print("--- Valori Mancanti dopo l'Imputazione ---")
print(df.isnull().sum())

# --- Statistiche Descrittive ---
print("\n--- Statistiche Descrittive dei Dati Puliti ---")
# Mostra le statistiche descrittive per una selezione di feature importanti
print(df[['BALANCE', 'PURCHASES', 'CASH_ADVANCE', 'CREDIT_LIMIT', 'PAYMENTS', 'TENURE']].describe())

--- Valori Mancanti dopo l'Imputazione ---
CUST_ID                             0
BALANCE                             0
BALANCE_FREQUENCY                   0
PURCHASES                           0
ONEOFF_PURCHASES                    0
INSTALLMENTS_PURCHASES              0
CASH_ADVANCE                        0
PURCHASES_FREQUENCY                 0
ONEOFF_PURCHASES_FREQUENCY          0
PURCHASES_INSTALLMENTS_FREQUENCY    0
CASH_ADVANCE_FREQUENCY              0
CASH_ADVANCE_TRX                    0
PURCHASES_TRX                       0
CREDIT_LIMIT                        0
PAYMENTS                            0
MINIMUM_PAYMENTS                    0
PRC_FULL_PAYMENT                    0
TENURE                              0
dtype: int64

--- Statistiche Descrittive dei Dati Puliti ---
        BALANCE  PURCHASES  CASH_ADVANCE  CREDIT_LIMIT  PAYMENTS   TENURE
count  8950.000   8950.000      8950.000      8950.000  8950.000 8950.000
mean   1564.475   1003.205       978.871      4494.282  1733.14

**Riepilogo della Pulizia dei Dati:**

*   I valori mancanti in `MINIMUM_PAYMENTS` e `CREDIT_LIMIT` sono stati imputati con successo utilizzando le rispettive mediane.
*   Le statistiche descrittive rivelano che la maggior parte delle variabili finanziarie (`BALANCE`, `PURCHASES`, `CASH_ADVANCE`) presenta una forte asimmetria positiva (media > mediana), indicando la presenza di outlier con valori elevati, un fenomeno tipico dei dati finanziari. Ciò conferma che sarà necessario un metodo di scalatura robusto.

### **3. Feature Scaling e Riduzione della Dimensionalità (PCA)**

Per preparare i dati per l'algoritmo K-Means, che è sensibile alla scala delle feature, applichiamo il `RobustScaler`. Questo metodo viene preferito allo `StandardScaler` per la sua robustezza agli outlier, una caratteristica nota del nostro dataset.

Dopo la scalatura, eseguiamo l'Analisi delle Componenti Principali (PCA). La PCA aiuta a ridurre la dimensionalità, mitigare la multicollinearità e trasformare i dati in un nuovo set di componenti non correlate che catturano la varianza più significativa. Ciò consente di ottenere un clustering più stabile e una visualizzazione più semplice.

In [ ]:
# Escludi l'ID cliente non numerico per la scalatura e la modellazione
X = df.drop('CUST_ID', axis=1)
feature_names = X.columns

# Utilizza RobustScaler in quanto è meno sensibile agli outlier
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# --- Analisi delle Componenti Principali (PCA) ---
# Applica la PCA per ridurre le dimensioni e catturare i principali pattern di varianza
pca = PCA(n_components=17)
X_pca = pca.fit_transform(X_scaled)

# --- Visualizzazione e Interpretazione della PCA ---
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(2, 2)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, :])

# Grafico Scree
ax1.plot(range(1, len(explained_variance) + 1), explained_variance, 'bo-', linewidth=2)
ax1.set_title('Scree Plot', fontsize=15)
ax1.set_xlabel('Componente')
ax1.set_ylabel('Varianza Spiegata')
ax1.grid(True, linestyle='--', alpha=0.6)

# Grafico della Varianza Cumulativa
ax2.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'ro-', linewidth=2)
ax2.axhline(y=0.80, color='g', linestyle='--', label='80% Varianza')
ax2.axhline(y=0.90, color='orange', linestyle='--', label='90% Varianza')
ax2.set_title('Varianza Cumulativa', fontsize=15)
ax2.set_xlabel('Componente')
ax2.set_ylabel('Varianza Cumulativa')
ax2.legend()
ax2.grid(True, linestyle='--', alpha=0.6)

# Heatmap della Matrice dei Loadings
loadings = pd.DataFrame(pca.components_[:].T, columns=[f'PC{i+1}' for i in range(17)], index=feature_names)
sns.heatmap(loadings, annot=True, cmap='viridis', fmt='.2f', ax=ax3)
ax3.set_title('Matrice dei Loadings', fontsize=15)

plt.tight_layout()
plt.show()

# Interpreta le componenti principali
print("\n--- Interpretazione dei Componenti Principali ---")
for i in range(17):
    print(f"\nPC{i+1} (Varianza: {explained_variance[i]:.2%}):")
    # Ordina i loadings per valore assoluto per visualizzare le feature più influenti
    sorted_loadings = loadings[f'PC{i+1}'].abs().sort_values(ascending=False).index
    print(loadings.loc[sorted_loadings, f'PC{i+1}'])

Dall'analisi PCA, si osserva che i primi componenti principali catturano una porzione significativa della varianza totale del dataset. Nello specifico:
- I primi 2 componenti spiegano circa il **54.82%** della varianza (30.26% + 24.56%).
- I primi 6 componenti spiegano circa l'**87.45%** della varianza, un livello considerato adeguato per la maggior parte delle analisi di clustering, in quanto bilancia la riduzione della dimensionalità con la conservazione delle informazioni.

L'interpretazione dei componenti principali più significativi rivela i principali assi di variabilità nel comportamento dei clienti:

*   **PC1 (Varianza: 30.26%): "Volume di Spesa e Pagamenti"**
    Questo componente è fortemente correlato positivamente con `MINIMUM_PAYMENTS`, `ONEOFF_PURCHASES`, `PURCHASES` e `PAYMENTS`. Un punteggio elevato su questo componente indica clienti con un alto volume di transazioni, sia in termini di acquisti (soprattutto in un'unica soluzione) che di pagamenti. Rappresenta l'intensità generale dell'attività finanziaria.

*   **PC2 (Varianza: 24.56%): "Comportamento di Pagamento: 'Revolver' vs. 'Transactor'"**
    PC2 mostra una forte correlazione positiva con `MINIMUM_PAYMENTS` e correlazioni negative con `ONEOFF_PURCHASES`, `PURCHASES` e `PRC_FULL_PAYMENT`. Questo componente distingue chiaramente due profili:
    *   **Punteggi alti (Revolvers):** Clienti che tendono a pagare solo l'importo minimo, mantenendo un saldo elevato.
    *   **Punteggi bassi (Transactors):** Clienti che effettuano acquisti significativi (spesso in un'unica soluzione) e pagano per intero il saldo, utilizzando la carta come strumento di transazione piuttosto che di credito.

*   **PC3 (Varianza: 12.32%): "Utilizzo dell'Anticipo Contanti"**
    Questo componente è dominato da `CASH_ADVANCE` e `CASH_ADVANCE_TRX`. Un punteggio elevato indica un uso intensivo del servizio di anticipo contanti, segnalando un profilo di cliente che utilizza la carta di credito come fonte di liquidità a breve termine, spesso associato a un rischio più elevato.

*   **PC4 (Varianza: 8.02%): "Frequenza di Aggiornamento e Attività del Conto"**
    Dominato da una forte correlazione positiva con `BALANCE_FREQUENCY`. Questo asse distingue i clienti che mantengono e aggiornano attivamente il loro saldo da quelli con un utilizzo più sporadico o dormiente della carta.

Questi primi componenti trasformano i dati originali in dimensioni di più facile interpretazione, che descrivono i principali archetipi comportamentali dei clienti, come l'intensità di spesa, la gestione del debito e l'uso di servizi specifici come l'anticipo contanti.

### **4. Determinazione del Numero Ottimale di Cluster (k)**

Per identificare il numero ottimale di cluster in cui segmentare la clientela, sono state applicate diverse metriche di validazione.

L'analisi è stata condotta sui **primi 6 componenti principali**, che catturano l'**87.45%** della varianza totale del dataset.

In [ ]:
# Utilizziamo i primi 6 componenti principali
n_components = 6
X_pca_reduced = X_pca[:, :n_components]

# Calcolo delle metriche per un range di cluster (da 2 a 10)
k_range = range(2, 11)
inertia = []
silhouette_scores = []
calinski_harabasz_scores = []
davies_bouldin_scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_pca_reduced)

    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_pca_reduced, kmeans.labels_))
    calinski_harabasz_scores.append(calinski_harabasz_score(X_pca_reduced, kmeans.labels_))
    davies_bouldin_scores.append(davies_bouldin_score(X_pca_reduced, kmeans.labels_))

# Visualizzazione dei risultati
fig, axs = plt.subplots(2, 2, figsize=(20, 15))
fig.suptitle('Metriche di Validazione del Clustering', fontsize=20)

# Elbow Method
axs[0, 0].plot(k_range, inertia, 'bo-', linewidth=2)
axs[0, 0].set_title('Elbow Method (Inertia)', fontsize=15)
axs[0, 0].set_xlabel('Numero di Cluster (k)')
axs[0, 0].set_ylabel('Inertia')
axs[0, 0].grid(True, linestyle='--', alpha=0.6)

# Silhouette Score
axs[0, 1].plot(k_range, silhouette_scores, 'go-', linewidth=2)
axs[0, 1].set_title('Silhouette Score', fontsize=15)
axs[0, 1].set_xlabel('Numero di Cluster (k)')
axs[0, 1].set_ylabel('Silhouette Score')
axs[0, 1].grid(True, linestyle='--', alpha=0.6)

# Calinski-Harabasz Index
axs[1, 0].plot(k_range, calinski_harabasz_scores, 'ro-', linewidth=2)
axs[1, 0].set_title('Calinski-Harabasz Index', fontsize=15)
axs[1, 0].set_xlabel('Numero di Cluster (k)')
axs[1, 0].set_ylabel('Calinski-Harabasz Score')
axs[1, 0].grid(True, linestyle='--', alpha=0.6)

# Davies-Bouldin Index
axs[1, 1].plot(k_range, davies_bouldin_scores, 'yo-', linewidth=2)
axs[1, 1].set_title('Davies-Bouldin Index', fontsize=15)
axs[1, 1].set_xlabel('Numero di Cluster (k)')
axs[1, 1].set_ylabel('Davies-Bouldin Score')
axs[1, 1].grid(True, linestyle='--', alpha=0.6)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

**Analisi dei Risultati di Validazione**

1.  **Elbow Method:** Il grafico dell'inerzia mostra un "gomito" pronunciato a k=3, con altri due scalini a k=6 e k=8.

2.  **Silhouette Score:** Questa metrica, che misura la separazione tra i cluster, raggiunge il suo valore massimo a k=2. Un punteggio più alto indica cluster ben definiti e distinti tra loro.

3.  **Calinski-Harabasz Index:** Questo indice, che valuta il rapporto tra la dispersione inter-cluster e intra-cluster, presenta un picco a k=3 e raggiunge il suo valore massimo a k=8. Valori più alti indicano una migliore qualità del clustering.

4.  **Davies-Bouldin Index:** Questa metrica, che misura la somiglianza media tra ciascun cluster e il suo più simile, presenta un picco di minimo a k=3 e raggiunge il suo valore più basso a k=8. Un punteggio più basso significa una migliore separazione tra i cluster.

**Conclusione sulla Validazione**

- **k=2**: Ottimo silhouette score (cluster molto separati), ma troppo semplicistico: due soli gruppi non colgono la complessità dei comportamenti di spesa e pagamento. Poco utile in chiave marketing mirato, perché ridurrebbe la personalizzazione a due macro-segmenti.

- **k=3**: Supportato dal “gomito” dell’Elbow Method, con buoni risultati da Calinski-Harabasz e Davies-Bouldin (minimi locali).
Mantiene una forte interpretabilità (equilibrio ottimale tra qualità dei cluster e semplicità).
Tre segmenti chiari, facilmente traducibili in strategie di marketing (es. spender occasionali, spender a rate, heavy users).

- **k=6–8**: Ottimizzano le metriche statistiche (CH max a 8, DB min a 8) ma il Silhouette score resta basso (cluster più numerosi ma meno netti).
Utile se si desidera una segmentazione molto granulare (es. micro-targeting con offerte super-personalizzate).
Maggiore dettaglio ma rischio di frammentazione e overfitting operativo per il marketing.

**Scelta Consigliata**

Per una prima segmentazione robusta, interpretabile e azionabile in termini di marketing **k=3** è l’opzione migliore.

Per scenari di hyper-personalization o campagne sofisticate basate su big data e automazione (es. recommendation engine, offerte dinamiche via app) **k=8** può essere adottato come livello di dettaglio superiore.

In ottica di ottimizzazione ROI e personalizzazione senza complicare eccessivamente l’operatività, la scelta più bilanciata è **k=3**.

In [ ]:
# Clustering finale
optimal_k = 3

# Usiamo i primi 6 componenti principali in grado di spiegare 87.45% della varianza.
X_analysis = X_pca[:, :6]

kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=20, max_iter=500)
cluster_labels = kmeans_final.fit_predict(X_analysis)

# DataFrame con le prime 6 componenti + cluster
pca_df = pd.DataFrame(X_pca[:, :6], columns=[f'PC{i+1}' for i in range(6)])
pca_df['Cluster'] = cluster_labels

# Pairplot
sns.pairplot(
    pca_df,
    hue="Cluster",
    palette="viridis",
    diag_kind="kde",
    plot_kws={'alpha':0.6, 's':30}
)
plt.suptitle(
    "Scatter Matrix delle Prime 6 Componenti Principali",
    y=1.02,
    fontsize=16
)
plt.show()

**Interpretazione dei Cluster e Profilazione dei Clienti**

Per comprendere le caratteristiche di ciascun cluster, analizziamo i valori medi delle variabili originali per ogni gruppo, confrontandoli con la media generale del dataset.

In [ ]:
# L'array `cluster_labels` contiene l'assegnazione del cluster per ogni cliente.
# Lo aggiungiamo come una nuova colonna chiamata 'CLUSTER' al nostro DataFrame principale.
df['CLUSTER'] = cluster_labels

# Analisi dei centroidi dei cluster con k=3
cluster_analysis_3 = df.drop(['CUST_ID'], axis=1).groupby('CLUSTER').mean()

# Calcolo della media generale per confronto
overall_mean = df.drop(['CUST_ID', 'CLUSTER'], axis=1).mean()

# Aggiungi la media generale come riga per un facile confronto
cluster_analysis_3.loc['Overall_Mean'] = overall_mean

# Stampa della tabella trasposta per una migliore leggibilità
print("--- Analisi dei Centroidi dei Cluster (k=3) ---")
print(cluster_analysis_3.T)

### **5. Profili dei Cluster e Personas**

**1) Cluster 0: Clienti Cauti e Basso Utilizzo**

**Comportamento**
*   **Saldo medio in linea con la media** (`BALANCE` ≈ 1449 vs media 1564), non accumulano debito significativo.
*   **Volume di acquisti molto basso** (`PURCHASES` ≈ 641, molto al di sotto della media di 1003).
*   **Poche transazioni di acquisto** (`PURCHASES_TRX` ≈ 11 vs media 15).
*   Utilizzo dell'anticipo contante in linea con la media (`CASH_ADVANCE` ≈ 958).
*   **Propensione media a pagare il saldo per intero** (`PRC_FULL_PAYMENT` ≈ 0.145, quasi identico alla media di 0.154).

**Persona**
Cliente **prudente e a basso rischio**. Utilizza la carta in modo sporadico, principalmente per piccole spese o per necessità di contante, senza però fare un uso eccessivo del credito. È un cliente solido e affidabile, ma poco profittevole per la banca data la scarsa attività.

---

**2) Cluster 1: "Revolvers" ad Alto Rischio**

**Comportamento**
*   **Saldo medio elevatissimo** (`BALANCE` ≈ 4569, quasi il triplo della media), indicativo di un forte accumulo di debito.
*   **Acquisti focalizzati su rateizzazioni** (`INSTALLMENTS_PURCHASES` ≈ 1106), mostrando una tendenza a finanziare le spese.
*   **Alto utilizzo dell'anticipo contante** (`CASH_ADVANCE` ≈ 1393, ben sopra la media).
*   **Pagamento del saldo completo quasi inesistente** (`PRC_FULL_PAYMENT` ≈ 0.003).
*   **Pagamenti minimi anomali ed estremamente alti** (`MINIMUM_PAYMENTS` ≈ 22307), segnale di un debito consolidato e molto ingente.

**Persona**
Cliente ad **alto rischio**, che utilizza la carta come principale fonte di credito per finanziare il proprio stile di vita. Fa molto affidamento su anticipi di contante e pagamenti rateali, ma è incapace di saldare il debito. Altamente profittevole nel breve termine per via degli interessi, ma con un **elevato rischio di default**.

---

**3) Cluster 2: Spender Elevati e Clienti "Premium"**

**Comportamento**
*   **Volume di acquisti eccezionale** (`PURCHASES` ≈ 7001, sette volte la media), trainato da acquisti in un'unica soluzione (`ONEOFF_PURCHASES` ≈ 4831).
*   **Frequenza d'uso molto alta** (`PURCHASES_FREQUENCY` ≈ 0.92 e `PURCHASES_TRX` ≈ 76), la usano costantemente.
*   **Limite di credito molto elevato** (`CREDIT_LIMIT` ≈ 9118, il doppio della media).
*   **Elevata capacità di pagamento** (`PAYMENTS` ≈ 7504), quasi pari al volume di acquisti.
*   **Buona propensione a pagare il saldo interamente** (`PRC_FULL_PAYMENT` ≈ 0.322, più del doppio della media).

**Persona**
Il **cliente ideale** per la banca: alto spendente, affidabile e redditizio. Utilizza la carta come strumento primario per spese importanti, genera un alto volume di transazioni e dimostra una solida capacità di ripagare il debito. Rappresenta il segmento più desiderabile, con alto potenziale di profitto e rischio gestibile.

---

**Sintesi Strategica per la Banca**

*   **Cluster 0 - Clienti Cauti**: Segmento "dormiente" a basso rischio.

**Strategie**: stimolare l'utilizzo con **promozioni mirate, cashback su categorie di spesa specifiche e campagne di engagement** per aumentarne la profittabilità.

*   **Cluster 1 - "Revolvers" ad Alto Rischio**: Segmento critico da monitorare attentamente.

**Strategie**: **piani di rientro personalizzati, blocco temporaneo di ulteriori anticipi di contante e consulenza finanziaria** per mitigare il rischio di insolvenza.

*   **Cluster 2 - Spender "Premium"**: Segmento di maggior valore.

**Strategie**: **programmi di fidelizzazione esclusivi, aumento del limite di credito, accesso a benefit premium (es. lounge aeroportuali, assicurazioni) e cross-selling** di altri prodotti finanziari.

#### **Azioni Consigliate per Cluster**

| Cluster | Profilo | Obiettivo Strategico | Azioni di Marketing e Gestionali Consigliate |
| :--- | :--- | :--- | :--- |
| **Cluster 0** | **Clienti Cauti** | **Attivazione e Engagement** | <ul><li>**Campagne di Cashback Mirate:** Offrire cashback su categorie di spesa quotidiana (es. supermercati, carburante) per incentivare l'uso abituale della carta.</li><li>**Promozioni su Acquisti Rateali:** Proporre piani "Compra Ora, Paga Dopo" a tasso zero per acquisti di importo medio-basso, per familiarizzare il cliente con le opzioni di credito.</li><li>**Comunicazioni Educative:** Inviare tutorial e guide su come sfruttare al meglio i vantaggi della carta (es. assicurazioni viaggio, sconti partner).</li></ul> |
| **Cluster 1** | **"Revolvers" ad Alto Rischio** | **Mitigazione del Rischio** | <ul><li>**Monitoraggio Proattivo:** Implementare alert automatici per il team di gestione del rischio quando il saldo supera soglie critiche.</li><li>**Piani di Rientro Personalizzati:** Offrire proattivamente piani di consolidamento del debito a un tasso di interesse inferiore per rendere sostenibile il rimborso.</li><li>**Limitazioni Selettive:** Valutare la sospensione temporanea della funzione di anticipo contante per i clienti con i profili di rischio più elevati.</li><li>**Consulenza Finanziaria:** Proporre sessioni di consulenza gratuita per aiutare i clienti a gestire meglio le proprie finanze.</li></ul> |
| **Cluster 2** | **Spender "Premium"** | **Fidelizzazione e Massimizzazione del Valore** | <ul><li>**Programmi di Loyalty Esclusivi:** Offrire l'upgrade a carte di credito di livello superiore (es. Black, Platinum) con benefit esclusivi come accesso a lounge aeroportuali, concierge personale e assicurazioni potenziate.</li><li>**Aumento del Limite di Credito:** Proporre proattivamente un aumento del fido per supportare il loro elevato volume di spesa.</li><li>**Opportunità di Cross-Selling:** Essendo clienti finanziariamente solidi, sono i candidati ideali per altri prodotti come prestiti personali, mutui, e servizi di wealth management.</li></ul> |

---

#### **Riepilogo del Progetto**
Questo progetto si è posto l'obiettivo di segmentare la clientela di FinTech Solutions S.p.A. per ottimizzare le strategie di marketing. Attraverso un processo di analisi esplorativa, preprocessing dei dati, riduzione della dimensionalità con PCA e applicazione dell'algoritmo di clustering K-Means, abbiamo identificato con successo **tre distinti segmenti di clientela**: i "Clienti Cauti", i ""Revolvers" ad Alto Rischio" e gli "Spender Premium". La scelta di tre cluster si è rivelata ottimale, bilanciando la significatività statistica con l'interpretabilità e l'azionabilità per il business.

#### **Valore Aggiunto per il Business**
L'adozione di questa segmentazione porterà benefici tangibili e strategici:
1.  **Massimizzazione del ROI delle Campagne:** Passando da comunicazioni di massa a messaggi personalizzati per ciascun cluster, l'azienda può **aumentare i tassi di conversione** e l'efficacia delle sue iniziative di marketing.

2.  **Gestione Proattiva del Rischio:** L'identificazione tempestiva del cluster ad alto rischio permette di implementare strategie mirate a **ridurre le perdite su crediti** e a **prevenire l'insolvenza**.

3.  **Aumento della Customer Lifetime Value:** Le strategie di fidelizzazione e cross-selling dedicate al segmento "Premium" permetteranno di **consolidare il rapporto con i clienti più profittevoli**, massimizzandone il valore nel lungo periodo.

4.  **Sviluppo di Prodotti Mirati:** La comprensione profonda dei diversi comportamenti d'uso può guidare lo **sviluppo di nuovi prodotti e servizi finanziari** che rispondano alle esigenze specifiche di ciascun segmento.